# ML workflow for the house prices dataset

This script contains a machine learning workflow for the house prices dataset. The workflow is based on scikit-learn pipelines. Different model architectures are used, including xgb, random forest, lgbm and ridge regression. Following grid search, models are combined in an ensemble to provide the final estimate.

In [28]:
# load libraries
import pandas as pd
import numpy as np
import pickle
import warnings
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Print file names in directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/competitions/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/competitions/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/competitions/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/competitions/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/eda-results/eda_config.pickle


In [5]:
# define necessary functions
def encode_ordinal(df, columns, order):
    for column in columns:
        df[column] = (
            pd.Categorical(df[column], categories=order, ordered=True)
            .codes
            .astype('float')
        )
        df[column] = df[column].replace(-1, np.nan)
    return df

# define function to run parameter grid search together with cross-validation
# important to specify scoring to negative RMSE (in line with kaggle)
def run_grid_search(estimator, param_grid, preprocessor, X, y, cv, scoring="neg_root_mean_squared_error"):
    
    # define pipeline
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    # set-up grid search
    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        cv=cv, # k-fold variable must be defined before so random state is always the same
        scoring=scoring,
        return_train_score=True
    )

    # run grid search
    grid.fit(X, y)

    # Extract results across runs (this runs independent of how many parameters are specified)
    results_df = pd.DataFrame({
        key.replace("param_model__", ""): grid.cv_results_[key]
        for key in grid.cv_results_.keys()
        if key.startswith("param_model__")
    })

    results_df["mean_rmse"] = -grid.cv_results_["mean_test_score"]  # Negate so that the final score is shown as positive
    results_df["std_rmse"] = grid.cv_results_["std_test_score"]

    return grid, results_df.sort_values("mean_rmse")

## Step 1: Load data and EDA results

In [6]:
# specify input paths
train_file_path = '/kaggle/input/competitions/house-prices-advanced-regression-techniques/train.csv'
test_file_path = '/kaggle/input/competitions/house-prices-advanced-regression-techniques/test.csv'
submission_file_path = '/kaggle/input/competitions/house-prices-advanced-regression-techniques/sample_submission.csv'
eda_config_file_path = '/kaggle/input/eda-results/eda_config.pickle'

# load as pandas data frames
df = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)
sub_df = pd.read_csv(submission_file_path)

# load eda config file
with open(eda_config_file_path, 'rb') as handle:
    eda = pickle.load(handle)

# separate variables
target = df["SalePrice"]
df = df.drop(columns=["SalePrice"])

# display options
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [7]:
# show dict keys of eda results
for i in eda.keys():
    print(i)

dropped_columns
ordinal_maps
dtype_overrides
category_missing_levels
final_features
numeric_features
categorical_features
rare_category_features
skewed_features
outlier_sensitive_features
target_transform
numeric_predictors_correlation
categorical_predictors_etasquared
high_corr_pairs_numeric
high_association_pairs_categorical


First, the dataframe will be corrected based on the results of the exploratory data analysis.

In [8]:
# recode into ordinal variables
for col, mapping in eda['ordinal_maps'].items():
    df = encode_ordinal(df, [col], mapping)
    df_test = encode_ordinal(df_test, [col], mapping)

# fix dtype
for col, dtype in eda['dtype_overrides'].items():
    df[col] = df[col].astype(dtype)
    df_test[col] = df_test[col].astype(dtype)

# log-transform target
target = np.log1p(target) # log-transformation takes care of the usual right-skew of price distributions

## Step 2: Pipeline set-up

The EDA previously identified several skewed numeric variables with outliers. Categorical variables were also shown to have some rare categories. The preprocessing pipeline can take this into account by scaling numeric variables and combining infrequent categories. Collapsing infrequent categories has the benefit of reducing the required columns for one-hot-encoding.

In [9]:
# split into numeric and categorical
numeric_features = eda['numeric_features']
categorical_features = eda['categorical_features']

In [10]:
# preprocessing numerical features (insert median for missing values and use z-score scaling)
numeric_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')), # impute the median for missing values
    ('scaler',  QuantileTransformer(output_distribution='normal', n_quantiles=1000)) # this ensures outliers and skew are minimized
])

# preprocessing categorical features (insert most frequent for missing values)
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')), # impute most frequent category if missing
    ('onehot', OneHotEncoder( # this is set up so that rare categories are grouped into a 'other' category
        handle_unknown = 'infrequent_if_exist',
        min_frequency = 0.01)) # this takes care of rare categories that appear less than 1%
])

In [11]:
# define the columns transformer
preprocessor = ColumnTransformer(transformers = [
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# prepare the train and test datasets
X = df[numeric_features + categorical_features]
X_test = df_test[numeric_features + categorical_features]

## Step 3: XGB regressor: Model training and cross-validation

The EDA has shown that there are a few pairs of variables with high correlation. Therefore, I will first use tree-based models since they are robust to multicollinearity.

In [12]:
# set random seed
rs = 42
np.random.seed(rs)

# define kfold split
kf = KFold(n_splits=5, shuffle=True, random_state=rs)

param_grid = {
    'model__max_depth': [3, 6, 9, 12],
    'model__n_estimators': [150, 250],
    'model__learning_rate': [0.05, 0.1]
}

estimator = XGBRegressor(random_state=rs)

grid_xgb, results_xgb = run_grid_search(
    estimator, param_grid, preprocessor, X, target, kf,
    scoring="neg_root_mean_squared_error"
)
results_xgb.head()

,learning_rate,max_depth,n_estimators,mean_rmse,std_rmse
9,0.10,3,250,0.131301,0.016746
1,0.05,3,250,0.132043,0.018642
8,0.10,3,150,0.132402,0.018134
0,0.05,3,150,0.135174,0.018080
3,0.05,6,250,0.138810,0.017403


In [13]:
# get the parameters with best RMSE estimate
best_max_depth = results_xgb['max_depth'].iloc[0]
best_n_estimators = results_xgb['n_estimators'].iloc[0]
best_learning_rate = results_xgb['learning_rate'].iloc[0]

# define the pipeline and model with these parameters
model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(
        random_state=rs,
        max_depth=best_max_depth,
        n_estimators=best_n_estimators,
        learning_rate=best_learning_rate
    ))
])

# train the model and predict test data
model.fit(X,target)
target_pred = model.predict(X_test)

In [14]:
# create results dataframe
submission = pd.DataFrame({
    'Id': df_test['Id'],
    'SalePrice': np.expm1(target_pred) # take inverse of logarithm for kaggle competition
})

# save in .csv format
submission.to_csv("submission_XGBRegressor.csv", index=False)

The basic XGB Regressor has a test RMSE of 0.133 (kaggle leaderboard). This estimate is very similar to the cross-validated RMSE, so no overfitting. However, there's several things left to do to try and increase the accuracy of the predictions. 

## Step 4: Random forest, LGBM, and ridge regression

Let's run several other models including the very similar random forest regressor as well as LGBM and the linear ridge estimator. These models probably won't be much better than XGB, but it might be helpful to combine these into an ensemble later, especially the ridge regressor.

In [15]:
param_grid = {
    'model__max_depth': [3, 6, 9, 12],
    'model__n_estimators': [100, 200, 250]
}

estimator = RandomForestRegressor(random_state=rs)

grid_rf, results_rf = run_grid_search(
    estimator, param_grid, preprocessor, X, target, kf,
    scoring="neg_root_mean_squared_error"
)
results_rf.head()

,max_depth,n_estimators,mean_rmse,std_rmse
10,12,200,0.143753,0.019226
11,12,250,0.143780,0.019167
9,12,100,0.144824,0.019361
7,9,200,0.145216,0.018387
8,9,250,0.145398,0.018423


In [16]:
param_grid = {
    'model__max_depth': [3, 6, 9, 12],
    'model__n_estimators': [100, 200, 250],
    'model__learning_rate': [0.05, 0.1]
}

estimator = LGBMRegressor(
    random_state=rs,
    verbose=-1,
    n_jobs=-1,
    enable_categorical=True
)

warnings.filterwarnings("ignore", message="X does not have valid feature names") # suppress error message, regressor will still run

grid_lgbm, results_lgbm = run_grid_search(
    estimator, param_grid, preprocessor, X, target, kf,
    scoring="neg_root_mean_squared_error"
)
results_lgbm.head()

,learning_rate,max_depth,n_estimators,mean_rmse,std_rmse
13,0.10,3,200,0.131366,0.016623
14,0.10,3,250,0.131380,0.016428
2,0.05,3,250,0.132025,0.018784
1,0.05,3,200,0.132810,0.018581
15,0.10,6,100,0.132969,0.017890


In [24]:
param_grid = {
    'model__alpha': np.linspace(0.1,3,5)
}

estimator = Ridge(random_state=rs)

grid_rid, results_rid = run_grid_search(
    estimator, param_grid, preprocessor, X, target, kf,
    scoring="neg_root_mean_squared_error"
)
results_rid.head()

,alpha,mean_rmse,std_rmse
4,3.000,0.136459,0.022484
3,2.275,0.136784,0.022220
2,1.550,0.137214,0.021935
1,0.825,0.137830,0.021628
0,0.100,0.138914,0.021261


## Step 5: Ensemble learning

In [25]:
# combine all models into an ensemble
ensemble = VotingRegressor([
    ('xgb', grid_xgb.best_estimator_),
    ('rf', grid_rf.best_estimator_),
    ('lgbm', grid_lgbm.best_estimator_),
    ('ridge', grid_rid.best_estimator_)
])

# fit the model
ensemble.fit(X, target)

VotingRegressor(estimators=[('xgb',
                             Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                QuantileTransformer(output_distribution='normal'))]),
                                                                               ['LotFrontage',
                                                                                'LotArea',
                                                                                'OverallQual',
                                                                                'OverallCond',
                                                                                'YearBuilt',
                                                                                'YearRemodAdd',
                                                                                'MasVnrArea',
                                                                                'ExterQual',
                                                                                'ExterCond',
                                                                                'Bs...
                                                                                'LandContour',
                                                                                'Utilities',
                                                                                'LotConfig',
                                                                                'LandSlope',
                                                                                'Neighborhood',
                                                                                'Condition1',
                                                                                'Condition2',
                                                                                'BldgType',
                                                                                'HouseStyle',
                                                                                'RoofStyle',
                                                                                'RoofMatl',
                                                                                'Exterior1st',
                                                                                'Exterior2nd',
                                                                                'Foundation',
                                                                                'Heating',
                                                                                'CentralAir',
                                                                                'Electrical',
                                                                                'Functional',
                                                                                'GarageType',
                                                                                'GarageFinish',
                                                                                'PavedDrive',
                                                                                'SaleType',
                                                                                'SaleCondition'])])),
                                             ('model',
                                              Ridge(alpha=np.float64(3.0),
                                                    random_state=42))]))])

In [27]:
target_pred = ensemble.predict(X_test)

# create results dataframe
submission = pd.DataFrame({
    'Id': df_test['Id'],
    'SalePrice': np.expm1(target_pred) # take inverse of logarithm for kaggle competition
})

# save in .csv format
submission.to_csv("submission_ensemble.csv", index=False)

The test RMSE from the ensemble submission is 0.124, significantly better than XGB alone. However, instead of the voting regressor, let's try the stacking regressor.

In [33]:
stack = StackingRegressor([
    ('xgb', grid_xgb.best_estimator_),
    ('rf', grid_rf.best_estimator_),
    ('lgbm', grid_lgbm.best_estimator_),
    ('ridge', grid_rid.best_estimator_)
], final_estimator=Ridge())

# fit the model
stack.fit(X, target)

StackingRegressor(estimators=[('xgb',
                               Pipeline(steps=[('preprocessor',
                                                ColumnTransformer(transformers=[('num',
                                                                                 Pipeline(steps=[('imputer',
                                                                                                  SimpleImputer(strategy='median')),
                                                                                                 ('scaler',
                                                                                                  QuantileTransformer(output_distribution='normal'))]),
                                                                                 ['LotFrontage',
                                                                                  'LotArea',
                                                                                  'OverallQual',
                                                                                  'OverallCond',
                                                                                  'YearBuilt',
                                                                                  'YearRemodAdd',
                                                                                  'MasVnrArea',
                                                                                  'ExterQual',
                                                                                  'ExterCond',
                                                                                  '...
                                               feature_weights=None, gamma=None,
                                               grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=None, max_bin=None,
                                               max_cat_threshold=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None,
                                               max_depth=None, max_leaves=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               multi_strategy=None,
                                               n_estimators=None, n_jobs=None,
                                               num_parallel_tree=None, ...))

In [34]:
target_pred = stack.predict(X_test)

# create results dataframe
submission = pd.DataFrame({
    'Id': df_test['Id'],
    'SalePrice': np.expm1(target_pred) # take inverse of logarithm for kaggle competition
})

# save in .csv format
submission.to_csv("submission_stack.csv", index=False)

Using the stack slightly decreased the RMSE to 0.123. Ridge was used as the final regressor since XGB, for example, let to a much higher RMSE likely due to overfitting. So, the stack predicted sales prices with an average error of 13% (conversion of log-RMSE).

## To-do

Several things can be added to this workflow to improve the predictions and visualize the results:

- Extract feature importance
- Add feature engineering
- Clip outliers/extreme values in the predictions